In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import vjp.preprocess as preprocess
from vjp.zero_shot import OpenAiClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
DF_FILENAME = 'connected_components.parquet'
if os.path.exists(DF_FILENAME):
    print(f'Reading from {DF_FILENAME}...')
    df = pd.read_parquet(DF_FILENAME)
else:           # Generate on the go
    print('File not found, generating dataframe...')
    namespace = preprocess.Namespace()
    namespace.connected_component_tags = ('req', 'arg', 'claim', 'mot', 'dec')
    namespace.use_child_text_tag_names = ('mot', 'dec')
    namespace.level = preprocess.PreprocessingLevels.CONNECTED_COMPONENTS
    df = preprocess.preprocess(namespace)

df.head()

In [ ]:
zero_shot_classifier = OpenAiClassifier(os.environ["OPENAI_API_KEY"],
                                        features=['fact', 'req', 'arg',
                                                  'claim', 'mot', 'dec'])

In [ ]:
results = zero_shot_classifier.predict(df.to_dict('records'))
cm = confusion_matrix(df['label'].tolist(), results)
disp = ConfusionMatrixDisplay(confusion_matrix=cm).plot()